In [1]:
import numpy as np
import random
import pandas as pd
import math


In [2]:
for i in range(125):
    df = pd.read_excel(f'./finaldataset/googleclusterdata{i}.xlsx')
    
    if i==0:
        data= df.copy()

    else :
        data= pd.concat([data,df],axis=0,ignore_index=True)

    print(i)    

print(df)     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


MemoryError: Unable to allocate 638. MiB for an array with shape (3, 27863928) and data type int64

In [ ]:
data

In [ ]:
class jobreq:
    #S types of Jobs serviced at each cloud, 
    # each specified by a three-tuple < ms,gs,ds>    
    def __init__(self,cloud,ms,gs,ds,jobtype):
        #request for cloud
        self.cloud = cloud
        #type of required VM
        self.ms=ms
        # # of VMs required+
        self.gs=gs
        # SLA in terms of Response time
        self.ds=ds
        #job type S
        self.jobtype = jobtype    

In [ ]:
# def randomsamplingHTD(length,clouds,requireddata):
#     a=2.0
#     dist = np.random.power(a, size = length)
#     scaled_dist = np.round(dist*9).astype(int)
#     values = np.random.choice(scaled_dist,size=length)
#     sample = pd.DataFrame(values,columns=['cloud'],index = list(requireddata.index.values))
#     sample = pd.concat([requireddata,sample],axis=1)
    
#     return sample
    

In [ ]:
def jobgenerator(df,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM):
    requests = []
#     requireddata =df.loc[df['timeslot'] == timeslot+1]
    condition = df["timeslot"] == timeslot+1
    requireddata = df[condition].copy()
    requireddata['cloud'] = np.random.randint(10,size = requireddata.shape[0])
    
    for i in range(len(requireddata)):
        m1=requireddata['sizeram'].iloc[i]
        m2=requireddata['sizecpu'].iloc[i]
        m = f'{m1}{m2}'
        g = requireddata['gs'].iloc[i]
        d = requireddata['ds'].iloc[i]
        c = requireddata['cloud'].iloc[i]
        t = (4*possibleVMtypes.index(m))+(2*possiblenoofVM.index(g))+possibleDS.index(d)
        requests.append(jobreq(c,m,g,d,t))
    return requests

In [ ]:
class CSPs:
  
    def __init__(self,name,nMI,betaIT,cuspay,typeofServiceS=24):
        self.nMI=nMI
        self.name=name
        #buy-bid to be submitted to auctioneer    
        self.buybid={}
        #sell-bid to be submitted to auctioneer
        self.sellbid={}
         #queue maintained by cloud
        self.typeofServiceS=typeofServiceS
        self.quelen = [0 for i in range(typeofServiceS)]
        self.possibleVMtype = ['ss','sm','sl','ls','lm','ll']
        self.possibleDS = [150,300]
        self.possiblenoofVM = [1,2]
        self.V=4000000
        #virtual queue maintained by cloud
        self.vQue = [0 for i in range(typeofServiceS)]
        # # of jobs dropped
        self.D = [0 for i in range(typeofServiceS)]
        #cost of running the server
        self.betaIT=betaIT
#         self.cusPay = [0.02,0.02,0.02,0.04,0.04,0.04]
        self.cusPay = cuspay
        self.cMI = [30,30,30,10,10,10]
        self.allService = self.obtainServiceSdesc()
        self.maxNoofJobdrop=1000
        self.xi=cuspay
        self.pays = [0 for i in range(5)]
        self.totalpay = 0
        
        
    def obtainServiceSdesc(self):
        possibleServiceS ={}
        x=0
        for i in possibleVMtypes:
            for j in possiblenoofVM:
                for k in possibleDS:
                    possibleServiceS[x]=[i,j,k]
                    x=x+1     
        return possibleServiceS


    def initqueue(self,rsit):
        for i in rsit[self.name]:
            self.quelen[i.jobtype] = self.quelen[i.jobtype]+1
            self.updatepayoff(0,(i.gs*self.cusPay[self.possibleVMtype.index(i.ms)]))
                    
    def keyAddition(self,dicts,key,value):
        if key not in list(dicts.keys()):
            temp=[]
            temp.append(value)
            dicts[key]=temp
        else:
            dicts[key].append(value)
        return dicts
    
    
    def getBids(self):
        self.wsdashi={}
        for i in range(len(self.quelen)):        
            if self.quelen[i]==0:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],0)
                continue        
            else:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],(self.quelen[i]+self.vQue[i])/self.allService[i][1])
        
        self.sStarM={}      
        for key,value in self.wsdashi.items():
            index = (self.typeofServiceS/len(self.possibleVMtype))*self.possibleVMtype.index(key)+value.index(max(value))
            self.sStarM[key]=index
            self.buybid[key]=max(value)/self.V
            self.sellbid[key]=max(max(value)/self.V,self.betaIT/self.cMI[self.possibleVMtype.index(key)])
#         print("Buybid:",self.buybid) 
#         print("Sellbid:",self.sellbid)
#         print("sStar:",self.sStarM)
#         print("wsdashi:",self.wsdashi)

    
    def notosell(self,ind):
        return self.cMI[ind]*self.nMI[ind]
      
    def queueupdate(self,muSIJ,rsit):
        for services in range(len(muSIJ)):
            summation = sum(muSIJ[services][self.name])
            if summation >= self.quelen[services]:
                self.quelen[services]=0
            else:
                self.quelen[services] = self.quelen[services] - summation 
        self.initqueue(rsit)
        
    def vqueueupdate(self,muSIJ,csps):
        for services in range(len(muSIJ)):
            msInd=self.possibleVMtype.index(self.allService[services][0])
            ds = self.allService[services][2]
            gs=self.allService[services][1]
            
            if ds==150:
                epsilonS = 1000
            elif ds==300:
                epsilonS=500
            
            if self.quelen[services]==0:
                t=1
            else:
                t=0
            temp = self.vQue[services] + (1-t)*(epsilonS-sum(muSIJ[services][self.name])) - t*sum([(i.cMI[msInd]*i.nMI[msInd])/gs for i in csps])
            self.vQue[services]=max(0,temp)
            
            
    def dropjobs(self):
        for services in range(self.typeofServiceS):
            tempxi =self.xi[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1]
            if (self.quelen[services]+self.vQue[services]) > (self.V*tempxi):
                self.D[services]=self.D[services]+self.maxNoofJobdrop
                self.quelen[services]=max(0,self.quelen[services]-self.maxNoofJobdrop)
                self.vQue[services]=max(0,self.vQue[services]-self.maxNoofJobdrop)
                self.updatepayoff(3,tempxi*1000)
                            
    def updatepayoff(self,ind,pay):
        self.pays[ind]= self.pays[ind]+pay
        
    def provisioningcost(self,nmi):
        for keys,values in nmi.items():
            self.pays[2]= self.pays[2]+ self.betaIT*values[self.name]
            
    def totalpayment(self):
        self.totalpay = self.pays[0]+self.pays[1]-self.pays[2]-self.pays[3]-self.pays[4]
            
    def dropinend(self):
        for services in range(self.typeofServiceS):
#             print(self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])])
            self.pays[0]= self.pays[0]- (self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1])
    

        
        
        

In [ ]:
class auctioneer:
         
    Buybid={}
    Sellbid={}
    possibleVMtype = ['ss','sm','sl','ls','lm','ll']
    
    def __init__(self,CSP):
        self.csps = CSP
        
    def allbidsandasks(self):
        for i in self.possibleVMtype:
            self.Buybid[i]={}
            self.Sellbid[i]={}
            for c in self.csps:
                self.Buybid[i][c.name]=c.buybid[i]
                self.Sellbid[i][c.name]=c.sellbid[i]
    
    def printbids(self):
        print("Buybids:",self.Buybid)
        print("SellBids:",self.Sellbid)
        
    def sortBid(self):
        for key in self.Buybid:
            self.Buybid[key]=dict(sorted(self.Buybid[key].items(),reverse=True, key=lambda item: item[1]))
            
        for key in self.Sellbid:
            self.Sellbid[key]=dict(sorted(self.Sellbid[key].items(),key=lambda item: item[1]))
            
            
    def winnerDetermination(self):
        self.winningBuyer={}
        self.winningSeller={}
        for key,value in self.Buybid.items():
            self.winningBuyer[key]={}
            s=sorted(set(list(value.values())))
            self.winningBuyer[key][list(value.items())[0][0]]=s[-2]
        print("WinningBuyer:",self.winningBuyer)
        
        for key,value in self.Sellbid.items():
            setofwinningseller=[]
            clearingprice = list(self.winningBuyer[key].values())[0]
            clearingsellprice=0

            for keys,values in value.items():
                if values <= clearingprice:
                    setofwinningseller.append(keys)
                    clearingsellprice=values
                    
            if len(setofwinningseller) != 0:   
                setofwinningseller.pop()
            self.winningSeller[key]=[]
            self.winningSeller[key].append(setofwinningseller)
            self.winningSeller[key].append(clearingsellprice)            
        print("WinningSeller:",self.winningSeller)
            
        
        # of VMs sold from cloud j to cloud i
        
    def determineAlpha(self):
        self.alphaMIJ = {}
        for size in self.possibleVMtype:
            self.alphaMIJ[size]=[[0 for i in range(len(self.csps))] for j in range(len(self.csps))]
            temp =[0 for x in range(len(self.csps))]
            for i in range(len(self.csps)):
                if i in self.winningSeller[size][0]:
                    temp[i]=self.csps[i].notosell(self.possibleVMtype.index(size))
            self.alphaMIJ[size][list(self.winningBuyer[size].keys())[0]]= temp.copy()
#         print("# of VMs sold j->i alphaMIJ:",self.alphaMIJ)
    
    def determineprice(self):
        for keys,values in self.winningSeller.items():
            winnername = list(self.winningBuyer[keys].keys())[0]
            temp = 0
            if len(values[0])>0:
                for i in values[0]:
                        self.csps[i].pays[1]=self.csps[i].pays[1] + self.alphaMIJ[keys][winnername][i]*values[1]
                        temp = temp + self.alphaMIJ[keys][winnername][i]
                        
            self.csps[winnername].pays[4] = self.csps[winnername].pays[4]+ temp * self.winningBuyer[keys][winnername]
                    
            
    

In [ ]:
def determineMu(csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    
    muSIT = [[[0 for i in csps] for j in csps] for k in range(csps[0].typeofServiceS)]

    for key,value in auc.alphaMIJ.items():
        winningBuy=list(auc.winningBuyer[key].keys())
        winningSell=list(auc.winningSeller[key][0])
        servtype = int(csps[winningBuy[0]].sStarM[key])
        gs = possibleService[servtype][1]
        msInd= csps[0].possibleVMtype.index(key)
        value = [[x//gs for x in y] for y in value] 
        for i in csps:
            if i.name in winningBuy or i.name not in winningSell:
                if ((i.quelen[servtype]+i.vQue[servtype])/gs) > (i.V*i.betaIT/i.cMI[msInd]) :
                    value[i.name][i.name]=i.notosell(msInd)//gs
        muSIT[servtype]=value

    return muSIT

In [ ]:
def noOfactivatedServers(muSIT,csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    activeNMI={}
    sumAlpha={}

    for key,value in auc.alphaMIJ.items():
        temp=[0 for i in csps]
        sumAlpha[key]=[]
        for cloud in value:
            temp = [(temp[i]+cloud[i]) for i in range(len(csps))]
        sumAlpha[key]=temp.copy()    
        
    sumMuSII  = {} 
    mS = -1
    for servtype in range(len(muSIT)):
        if mS != possibleService[servtype][0]:
            mS=possibleService[servtype][0]
            sumMuSII[mS]=[]
            temp=[0 for i in csps]

        for j in range(len(muSIT[servtype])):
            temp[j] = temp[j]+muSIT[servtype][j][j]*possibleService[servtype][1]
            
        sumMuSII[mS]= temp.copy()
    
    for key,value in sumMuSII.items():
        activeNMI[key]=[(value[i]+sumAlpha[key][i])//csps[0].cMI[csps[0].possibleVMtype.index(key)] for i in range(len(csps)) ]
            
    return activeNMI       

In [ ]:
def auctionprocess(csps):
    auc = auctioneer(csps)
    auc.allbidsandasks()
    auc.sortBid()
#     auc.printbids()
    auc.winnerDetermination()
    auc.determineAlpha()
    auc.determineprice()
    return auc

In [ ]:
if __name__ =="__main__":
    print("    ROUND-0   \n")
    noofClouds = 10
    possibleVMtypes = ['ss','sm','sl','ls','lm','ll']
    possibleDS = [150,300]
    possiblenoofVM = [1,2]
    VMprice = [58.75,97,173.51,76.26,117.51,194] #values are in $/months(730 hours)
    VMpricepH = [ i/730 for i in VMprice]
    clouds = [i for i in range(noofClouds)]
    
    CSP = []

    for i in range(10):
        CSP.append(CSPs(i,np.random.randint(800,1000, size=6).tolist(),random.uniform(0.3,0.4),VMpricepH))
    

    timeslot = -1
    request = jobgenerator(data,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)
    
    
    
#resource request per round for s type of job
    rsit = [[] for i in CSP]

    for req in request: 
        rsit[req.cloud].append(req)   
    
    for c in CSP:
        c.initqueue(rsit)
        c.getBids()
        
    auc= auctionprocess(CSP)
    
    muSIT = determineMu(CSP,auc)
#     print("\n# of jobs i->j:",muSIT)
    active=noOfactivatedServers(muSIT,CSP,auc)  
#     print("\n# of Active Servers:",active)
    for i in CSP:
        i.queueupdate(muSIT,rsit)
   


    timeperiod = 2000
    for t in range(timeperiod):
        print(f"\n      ROUND-{t+1}    \n ")
        newtimeslot = t%696
        request = jobgenerator(data,newtimeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)

        rsit = [[] for i in CSP]

        for req in request: 
            rsit[req.cloud].append(req)   
    
        for c in CSP:
            c.getBids()

        auc= auctionprocess(CSP)
        
        muSIT = determineMu(CSP,auc)
#         print("\n# of jobs i->j:",muSIT)
        active=noOfactivatedServers(muSIT,CSP,auc)  
#         print("\n# of Active Servers:",active)
#updating the queue
        for i in CSP:
            i.dropjobs()
            i.vqueueupdate(muSIT,CSP)
            i.queueupdate(muSIT,rsit)
            i.provisioningcost(active)
            
    
    temp = {}        
    for i in CSP:
#         print("total no. of job drop: ",i.pays)
        
#         i.dropinend()
#         print("quuelen :",i.quelen)
#         print("total no. of job drop: ",i.pays)
        i.totalpayment()
        temp[f"CSP{i.name}"] = i.totalpay//timeperiod
        temp1[f"CSP{i.name}"] = i.D
#         print(f"total pay of {i.name} : ",i.totalpay//timeperiod)
#         print(f"total pay of {i.name} : ",i.totalpay)
        

    print(temp) 
    
    

In [ ]:
item = pd.DataFrame.from_dict(temp, orient='index')
item1 = pd.DataFrame.from_dict(temp1, orient='index')

writer = pd.ExcelWriter(f'./outputpay/paydata4.xlsx', engine='xlsxwriter')
item.to_excel(writer,sheet_name="pay of all clouds")
item1.to_excel(writer,sheet_name="no of job drop")
writer.save()

In [ ]:
# for i in CSP:
#     print(f"paysOff of ${i.name}:",i.pays)

In [ ]:
# for i in CSP:
#     print(i.D)



In [ ]:
# condition = data["timeslot"] < 101 
# req = data[condition]
# # print(df)
# condition =  req['sizecpu']=='s'
# req = req[condition].copy()
# condition =  req['sizeram']=='l'
# req = req[condition].copy()
# print(len(req))
